<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span><ul class="toc-item"><li><span><a href="#OS" data-toc-modified-id="OS-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>OS</a></span></li><li><span><a href="#Python" data-toc-modified-id="Python-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Python</a></span></li><li><span><a href="#mayavi" data-toc-modified-id="mayavi-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>mayavi</a></span><ul class="toc-item"><li><span><a href="#mayavi-install" data-toc-modified-id="mayavi-install-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>mayavi install</a></span></li></ul></li><li><span><a href="#numpy-quaternion" data-toc-modified-id="numpy-quaternion-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>numpy-quaternion</a></span></li><li><span><a href="#Optional:-inline-3d-plotting" data-toc-modified-id="Optional:-inline-3d-plotting-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Optional: inline 3d plotting</a></span></li></ul></li><li><span><a href="#bone-class" data-toc-modified-id="bone-class-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>bone class</a></span></li><li><span><a href="#Maths-functions" data-toc-modified-id="Maths-functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Maths functions</a></span><ul class="toc-item"><li><span><a href="#mag" data-toc-modified-id="mag-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>mag</a></span></li><li><span><a href="#angle" data-toc-modified-id="angle-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>angle</a></span></li></ul></li><li><span><a href="#Rotation" data-toc-modified-id="Rotation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Rotation</a></span></li><li><span><a href="#Table-of-Angles" data-toc-modified-id="Table-of-Angles-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Table of Angles</a></span></li><li><span><a href="#Plotting:" data-toc-modified-id="Plotting:-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Plotting:</a></span><ul class="toc-item"><li><span><a href="#bone_plot" data-toc-modified-id="bone_plot-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>bone_plot</a></span></li></ul></li><li><span><a href="#How-to-use:" data-toc-modified-id="How-to-use:-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>How to use:</a></span><ul class="toc-item"><li><span><a href="#1.-Set-the-root-directory-for-the-matlab-file-loader" data-toc-modified-id="1.-Set-the-root-directory-for-the-matlab-file-loader-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>1. Set the root directory for the matlab file loader</a></span></li><li><span><a href="#2.-Load-the-data-that-you-want-to-use" data-toc-modified-id="2.-Load-the-data-that-you-want-to-use-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>2. Load the data that you want to use</a></span></li><li><span><a href="#Rotate-bone" data-toc-modified-id="Rotate-bone-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Rotate bone</a></span></li><li><span><a href="#6.-Plotting-the-rotation" data-toc-modified-id="6.-Plotting-the-rotation-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>6. Plotting the rotation</a></span></li><li><span><a href="#Table-of-Angels" data-toc-modified-id="Table-of-Angels-7.5"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>Table of Angels</a></span></li></ul></li></ul></div>

In [1]:
import math
import numpy as np
import pandas as pd
import scipy.io
from pathlib import Path

from mayavi import mlab
import quaternion as quat
from sklearn.decomposition import PCA

# Libraries
See environment.yml 

conda env create -f environment.yml

conda activate sci


## OS
Has been written (and runs) on both Windows 10 and MacOS

## Python
This was written on python 3.6, python 2.7x versions won't work due to the f strings

## mayavi
This is the 3d plotting library used for rendering the plots. mayvai will launch a qt window to display the plot- meaning that you will need an X serve session for the plots to load. If you want to plot things inline you will need to use jupyter notebooks, not jupyter lab.

### mayavi install
https://docs.enthought.com/mayavi/mayavi/installation.html#installing-with-conda-forge


    
## numpy-quaternion
https://github.com/moble/quaternion
https://quaternion.readthedocs.io/en/latest/

    conda install -c conda-forge quaternion
    
    
## Optional: inline 3d plotting
http://docs.enthought.com/mayavi/mayavi/tips.html#using-mayavi-in-jupyter-notebooks

In [2]:
# Only works with notebooks not lab
#mlab.init_notebook('x3d', 500, 500)

In [3]:
# test inline rendering
#s = mlab.test_plot3d()
#s

# bone class

This class manages the loading of voxel objects and creating its atributes 

In [4]:
class bone:

    filter_level=0.001

    def __init__(self, array):
               
        """
        Performs calculations on the voxel array objects    
        array (np.array): binary voxel object)      
        filter_level (int/float): sets the threshold level for 
        what is considered a voxel. Everything below filter level is
        rounded to 0, everything above rounded to 1 (ie voxel)
        """ 
        self.array = array
        self.get_xyz()


    def get_xyz(self):
        """Convert 3D voxel array to xyz coordinates.
    
        array (np.array): 3D voxel array  
            
        filter_level (int/float): (inherited from `bone` class) sets the threshold level for 
        what is considered a voxel. Everything below filter level is
        rounded to 0, everything above rounded to 1 (ie voxel)
            
        returns: 
            np.array( [n x 3] )
    """
        # Everything above filter level is converted to 1
        filtered_array = np.where(self.array < self.filter_level, 0, 1)
        
        # records coordiates where there is a 1
        x, y, z = np.where(filtered_array == 1)

        self.xyz = np.array([x, y, z]).T


    def get_pca(self):
        """PCA on the xyz points array
    
            xyz(np.array): n x 3 array of xyz coordinates
            
            returns:    self.pc1
                        self.pc2
                        self.pc3 """
         
        pca = PCA(n_components=3, svd_solver='full')
        pca.fit_transform(self.xyz)

        self.PC1 = pca.components_[0]
        self.PC2 = pca.components_[1]
        self.PC3 = pca.components_[2]    



    def get_mean(self):
        """The mean of the xyz atriube
    
            returns:
                tupple (mean_of_x, mean_of_y ,mean_of_z)"""

        #mean_x, mean_y, mean_z
        return (np.mean(self.xyz[:,0]), np.mean(self.xyz[:,1]), np.mean(self.xyz[:,2]))
        

    def center_to_origin(self):
        """ sets the mean of the bone to 0,0,0"""    

        # set transformation (tfm) value
        self.tfm = self.get_mean()

        self.xyz = self.xyz - self.tfm


    def reset_position(self):
        """ resets the position of the bone to its orginal one"""
        self.xyz = self.xyz + self.tfm

    # Alternative constructor:
    # Import directly from matlab path

    @classmethod 
    def from_matlab_path(cls, root_dir, matlab_file):
        """Imports matlab file drectly
        
           path: path object/string 

           retruns np.array (n x n x n )
        """
        
        matlab_object = scipy.io.loadmat(root_dir/matlab_file)
        obj = matlab_object.keys()
        obj = list(obj)
        array = matlab_object[obj[-1]]
        
        return cls(array)


# Maths functions

## mag

In [5]:
def mag(v):
    """ Finds magnitude of vector

        v (np.array): vector
    """
    return math.sqrt(np.dot(v,v))

## angle

In [6]:
def angle(v1, v2):
    """ Finds angel between 2 vectors
    
    returns: ang , v1
    """
    try:
        ang = math.acos(np.dot(v1, v2) / (mag(v1) * mag(v2)))
   
        if ang >= math.pi/2:
            v1 = -v1
            ang = math.acos(np.dot(v1, v2) / (mag(v1) * mag(v2)))
            print(f'{ang} PC inverted')
        
        else:
            print(f'{ang} no invert')


    except:
        #ang = 0
        print (f'ERROR: angles are the same v1= {v1}, v2= {v2}')
        ang = 0
       
    return ang , v1

# Rotation

The bones are rotated with quaternions.

The angle between the two PC1 vectors is taken. The object is then rotated (by a quaternion) around the cross product between the PC1 vectors.

The new angles between the next PCs are calculates and the process is repeated for the other PCs

In [7]:
def quaternion_rotation(v, c_axis,theta):

    rotation_axis = np.array([0.] + c_axis)
    axis_angle = (theta*0.5) * rotation_axis/np.linalg.norm(rotation_axis)

    vec = quat.quaternion(*v)

    # quaternion from exp of axis angle
    qlog = quat.quaternion(*axis_angle)
    q = np.exp(qlog)

    # double cover quaternion rotation
    v_prime = q * vec * np.conjugate(q)

    return v_prime.imag

In [8]:
def voxel_rotate(bone_f1, bone_f2):

    #center bones too 0,0,0,
    bone_f1.center_to_origin()
    bone_f2.center_to_origin()

    # PCA on bones
    bone_f1.get_pca()
    bone_f2.get_pca()
    
    # for 1 to 3 principle conponents of the object
    for n in range(1,4):

        # takes cross product axis
        cross_product_axis = np.cross(
            getattr(bone_f1,f'PC{n}'),
            getattr(bone_f2,f'PC{n}'))         

        # finds angle between PCs for f1 vs f2
        theta, vector = angle(
            getattr(bone_f1,f'PC{n}'),
            getattr(bone_f2,f'PC{n}'))

        # sets any new values needed
        setattr(bone_f1,f'PC{n}',vector)

        #rotates each PC
        for n in range(1,4):
            transformed_PC = quaternion_rotation(
                v = getattr(bone_f1,f'PC{n}'),
                c_axis=cross_product_axis,
                theta=theta)

            #sets new PCA
            setattr(bone_f1, f'PC{n}',transformed_PC)

        #rotates xyz array
        rotated_xyz = np.apply_along_axis(
                    quaternion_rotation, 1, getattr(bone_f1,'xyz'),
                    c_axis=cross_product_axis, 
                    theta=theta)  

        setattr(bone_f1,'xyz',rotated_xyz)

# Table of Angles

In [9]:
def df_angles(bone_f1,bone_f2, name ='UN-NAMED BONE'):
    """
    Compares the PCA angles between to bones.
    
    Input:  bone_f1 = bone in 1st position
            bone_f2 = bone in 2nd position
               
    Returns: pandas dataframe
    """
    
    df = pd.DataFrame()

    # loops over each PCA
    for n in range(1,4):
        theta, _ = angle(getattr(bone_f1,f'PC{n}'),getattr(bone_f2,f'PC{n}'))

        # Sets the column names
        df.loc[f'{name} f1: PC{n}',f'{name}f2: PC{n}'] = theta
            
    return df

# Plotting:

Creates plots that show both the PCs and the voxelized bones

## bone_plot

This plots an arbitrary number of voxelized bones.

eg: `bone_plot(bone.bone_name.f1)`

`plot_PCA`: plots the PCAs as vectors on the bone

`plot_inv`: plots the inverse of each PCA so the axes go in both directions

You can use your own colours by passing a colour dictionary

`my_colours = {'red':(1,0,0),'green':(0,1,0),'blue':(0,0,1)`
         
The first bone will be plotted with the first colour in the dictionary, the second with the second and so on.


In [10]:
def bone_plot(*args, user_colours = None, plot_PCA = True, plot_inv = False):
    """ Plots voxel array that has an xyz attribute;
        can take n bones and plot PCA vectors
        PC1 Red 
        PC2 Blue
        PC3 Green

        plot_PCA: plots the PCAs as vectors on the bone
        plot_inv: plots the inverse of each PCA vector (PCAs go in both directions)
    """

    # Sorting out colours
    colour_dict = {'yellow':(0.9,0.9,0),
                   'pastel_blue':(0.7,1,1),
                   'purple':(0.6,0,0.5),
                   'orange':(0.8,0.3,0),
                   'dark_blue':(0,0.3,0.7),}
    
    if user_colours is None:
        user_colours = colour_dict
    
    plot_colours = []
    
    for col in user_colours:
        x = colour_dict.get(col)
        plot_colours.append(x)
        
    
    for n, bone in enumerate(args):

        mlab.points3d(bone.xyz[:,0], 
                      bone.xyz[:,1], 
                      bone.xyz[:,2],
                     mode="cube",
                     color= plot_colours[n],
                     scale_factor=1)
        
        x,y,z = bone.get_mean()

        #plot PCAs
        u0,v0,w0 = bone.PC1 * 100
        u0_inv,v0_inv,w0_inv = bone.PC1 * 100 * -1

        u1,v1,w1 = bone.PC2 * 100
        u1_inv,v1_inv,w1_inv = bone.PC2 * 100 * -1

        u2,v2,w2 = bone.PC3 * 100
        u2_inv,v2_inv,w2_inv = bone.PC3 * 100 * -1

        #print(f"{n}th bone PCA vectors: \n {bone.vec} \n ")
        
        
        if plot_PCA is True:
            mlab.quiver3d(x,y,z,u0,v0,w0,
                                 line_width =6,
                                 scale_factor=0.7,
                                 color= (1,0,0))
            mlab.quiver3d(x,y,z,u1,v1,w1,
                                 line_width =6,
                                 scale_factor= 0.5,
                                 color= (0,1,0))
            mlab.quiver3d(x,y,z,u2,v2,w2, 
                                 line_width =6,
                                 scale_factor=0.3,
                                 color=(0,0,1))


        #plotting the inverse of PCAs
        if plot_inv is True:
            mlab.quiver3d(x,y,z,u0_inv,v0_inv,w0_inv,
                                 line_width =6,
                                 scale_factor=0.7,
                                 color= (1,0,0))
            mlab.quiver3d(x,y,z,u1_inv,v1_inv,w1_inv,
                                 line_width =6,
                                 scale_factor=0.5,
                                 color= (0,1,0))
            mlab.quiver3d(x,y,z,u2_inv,v2_inv,w2_inv,
                                 line_width =6,
                                 scale_factor=0.3,
                                 color=(0,0,1))

    return mlab.show()

In [11]:
# Option user colours
# user_colours=['yellow','purple']

# How to use:

## 1. Set the root directory for the matlab file loader

In [12]:
root_dir = Path('C://Users/luke/OneDrive - University College London/Marta/data')

## 2. Load the data that you want to use

In [13]:
# load data phantoms
tibia_f2 = bone.from_matlab_path(root_dir, matlab_file = 'phantom/phantom_tibia_f2.mat' )
tibia_f1 = bone.from_matlab_path(root_dir, matlab_file = 'phantom/phantom_tibia_f1.mat')

## Rotate bone

In [14]:
voxel_rotate(tibia_f1,tibia_f2)

0.1747168069985997 no invert
0.5819453559235449 no invert
ERROR: angles are the same v1= [ 0.7214386  -0.69025737 -0.05541761], v2= [ 0.7214386  -0.69025737 -0.05541761]


## 6. Plotting the rotation

In [15]:
bone_plot(tibia_f1,tibia_f2)

## Table of Angels

In [16]:
df_angles(tibia_f1,tibia_f2, name= 'tibia')

0.0 no invert
1.4901161193847656e-08 no invert
ERROR: angles are the same v1= [ 0.7214386  -0.69025737 -0.05541761], v2= [ 0.7214386  -0.69025737 -0.05541761]


,tibiaf2: PC1,tibiaf2: PC2,tibiaf2: PC3
tibia f1: PC1,0.0,NaN,NaN
tibia f1: PC2,NaN,1.490116e-08,NaN
tibia f1: PC3,NaN,NaN,0.0
